## Filtering the OMOP Dataset

At this point in the dataset creation pipeline, the correct vocabulary standards have been applied. We now need to filter invalid records as well as narrow down the dataset to include only patient records that have a usable number of visit records. We decided to filter out any patient record that did not have more than 2 visit records.

### Filter description

Since all of the main clinical tables in OMOP have a person_id reference, we can use it to filter down all the tables. We iterate through all the clinical tables and only select records where the person to whom it belongs has three attributes:
    1. The recorded death date is greater than or equal to the largest visit date
    2. The recorded birth date is less than or equal to the smallest visit date
    3. The patient has at least three unique visits in the OMOP repository

In [2]:
def filter_dataset():
    tables = [  
        'person'
        'condition_occurrence',
        'death',
        'drug_exposure',
        'measurement',
        'observation',
        'observation_period',
        'procedure_occurrence',
        'visit_occurrence'
    ]
    
    for table in tables:
        query = f"""
            SELECT *
            FROM amalga.omop.{table} tab
            WHERE 
            NOT EXISTS
                (
                    SELECT p.person_id
                    FROM 
                        amalga.omop.visit_occurrence v 
                            RIGHT JOIN 
                        amalga.omop.person p 
                            ON p.person_id = v.person_id
                    WHERE tab.person_id=p.person_id
                    GROUP BY p.person_id, CAST(CAST(p.year_of_birth AS varchar) + '-' + CAST(p.month_of_birth AS varchar) + '-' + CAST(p.day_of_birth AS varchar) AS DATETIME)
                    HAVING MIN(visit_start_date) < CAST(CAST(p.year_of_birth AS varchar) + '-' + CAST(p.month_of_birth AS varchar) + '-' + CAST(p.day_of_birth AS varchar) AS DATETIME)
                ) AND
            NOT EXISTS
                (
                    SELECT d.person_id
                    FROM 
                        amalga.omop.visit_occurrence v 
                            RIGHT JOIN 
                        amalga.omop.death d 
                            ON d.person_id = v.person_id
                    WHERE tab.person_id=d.person_id
                    GROUP BY d.person_id, d.death_date
                    HAVING MAX(visit_start_date) > d.death_date
                ) AND
            NOT EXISTS
                (
                    SELECT v.person_id
                    FROM amalga.omop.visit_occurrence v
                    WHERE tab.person_id=v.person_id
                    GROUP BY v.person_id
                    HAVING COUNT( DISTINCT v.visit_occurrence_id) >= 3
                );
        """
        output = dbf.query(query)
        
        return output